In [6]:
import requests, math, time
import pandas as pd
import numpy as np

BASE = "https://clinicaltrials.gov/api/v2/studies"
params = {
    "format": "json",
    "pageSize": 100,
    "filter.overallStatus": "COMPLETED,TERMINATED,WITHDRAWN,SUSPENDED",
    "countTotal": "true"
}

In [39]:
def fetch_all():
    out = []
    page_token = None
    while True:
        p = params.copy()
        if page_token: 
            p["pageToken"] = page_token
        r = requests.get(BASE, params=p, timeout=60)
        print(r.url)                     # see the final URL
        print(r.status_code)
        if r.status_code >= 400:
            print(r.text[:800])  
        r.raise_for_status()
        data = r.json()
        studies = data.get("studies", [])
        out.extend(studies)
        page_token = data.get("nextPageToken")
        if not page_token: break
        time.sleep(0.2)  # be nice to the API
    return out
raw = fetch_all()
len(raw)


https://clinicaltrials.gov/api/v2/studies?format=json&pageSize=100&filter.overallStatus=COMPLETED%2CTERMINATED%2CWITHDRAWN%2CSUSPENDED&countTotal=true
200
https://clinicaltrials.gov/api/v2/studies?format=json&pageSize=100&filter.overallStatus=COMPLETED%2CTERMINATED%2CWITHDRAWN%2CSUSPENDED&countTotal=true&pageToken=ZVNj7o2Elu8o3lptUtuuqbL-mpOQJJxoYvep
200
https://clinicaltrials.gov/api/v2/studies?format=json&pageSize=100&filter.overallStatus=COMPLETED%2CTERMINATED%2CWITHDRAWN%2CSUSPENDED&countTotal=true&pageToken=ZVNj7o2Elu8o3lptUtuuqbL-mpOQJJxsZPSk0A
200
https://clinicaltrials.gov/api/v2/studies?format=json&pageSize=100&filter.overallStatus=COMPLETED%2CTERMINATED%2CWITHDRAWN%2CSUSPENDED&countTotal=true&pageToken=ZVNj7o2Elu8o3lptUtuuqbL-mpOQJJxrZvOj0A
200
https://clinicaltrials.gov/api/v2/studies?format=json&pageSize=100&filter.overallStatus=COMPLETED%2CTERMINATED%2CWITHDRAWN%2CSUSPENDED&countTotal=true&pageToken=ZVNj7o2Elu8o3lptUtuuqbL-mpOQJJxqZfWg0A
200
https://clinicaltrials.gov/api/

354712

In [3]:
print(len(raw))        # how many studies you got
print(type(raw))          # should be <class 'list'>
print(type(raw[0]))        # should be <class 'dict'>
print(raw[0].keys())

NameError: name 'raw' is not defined

In [9]:
import json
print(json.dumps(raw[0], indent=2))

NameError: name 'raw' is not defined

In [ ]:
def flatten_study(study):
    ps = study.get("protocolSection", {})
    id_mod = ps.get("identificationModule", {})
    status_mod = ps.get("statusModule", {})
    design_mod = ps.get("designModule", {})
    sponsor_mod = ps.get("sponsorCollaboratorsModule", {})

    return {
        "nct_id": id_mod.get("nctId"),
        "brief_title": id_mod.get("briefTitle"),
        "overall_status": status_mod.get("overallStatus"),
        "start_date": status_mod.get("startDateStruct", {}).get("date"),
        "completion_date": status_mod.get("completionDateStruct", {}).get("date"),
        "phase": ",".join(design_mod.get("phases", []) or []),
        "study_type": design_mod.get("studyType"),
        "lead_sponsor": sponsor_mod.get("leadSponsor", {}).get("name"),
    }


In [8]:
def flatten_study(study):
    ps = study.get("protocolSection", {}) or {}

    idm = ps.get("identificationModule", {}) or {}
    sm  = ps.get("statusModule", {}) or {}
    scm = ps.get("sponsorCollaboratorsModule", {}) or {}

    # dates may be nested in structs like {"date":"YYYY-MM", "type":"ACTUAL"}
    start_date       = (sm.get("startDateStruct") or {}).get("date")
    completion_date  = (sm.get("completionDateStruct") or {}).get("date")
    first_post_date  = (sm.get("studyFirstPostDateStruct") or {}).get("date")
    last_update_date = (sm.get("lastUpdatePostDateStruct") or {}).get("date")

    # collaborators is a list of dicts -> join names into one string
    collabs = scm.get("collaborators") or []
    collaborator_names = "|".join([c.get("name","") for c in collabs if c.get("name")])

    return {
        # identification
        "nct_id":        idm.get("nctId"),
        "org_study_id":  (idm.get("orgStudyIdInfo") or {}).get("id"),
        "brief_title":   idm.get("briefTitle"),
        "official_title":idm.get("officialTitle"),
        "org_full_name": (idm.get("organization") or {}).get("fullName"),
        "org_class":     (idm.get("organization") or {}).get("class"),

        # status / dates
        "overall_status":      sm.get("overallStatus"),
        "status_verified":     sm.get("statusVerifiedDate"),
        "start_date":          start_date,
        "completion_date":     completion_date,
        "first_submit_date":   sm.get("studyFirstSubmitDate"),
        "first_submit_qc":     sm.get("studyFirstSubmitQcDate"),
        "first_post_date":     first_post_date,
        "last_update_submit":  sm.get("lastUpdateSubmitDate"),
        "last_update_post":    last_update_date,
        "expanded_access":     (sm.get("expandedAccessInfo") or {}).get("hasExpandedAccess"),

        # sponsors / collaborators
        "lead_sponsor_name": (scm.get("leadSponsor") or {}).get("name"),
        "lead_sponsor_class":(scm.get("leadSponsor") or {}).get("class"),
        "collaborators":     collaborator_names,

        # top-level flags
        "has_results": bool(study.get("hasResults")),
    }


In [7]:
flat = [flatten_study(s) for s in raw]
import pandas as pd
df = pd.DataFrame(flat).reset_index(drop=True)


NameError: name 'raw' is not defined

In [11]:
df.to_csv("clinical_trials_clean.csv", index=False)

KeyboardInterrupt: 

In [17]:
df = pd.read_csv("clinical_trials_clean.csv")  
df.head()


,nct_id,org_study_id,brief_title,official_title,org_full_name,org_class,overall_status,status_verified,start_date,completion_date,first_submit_date,first_submit_qc,first_post_date,last_update_submit,last_update_post,expanded_access,lead_sponsor_name,lead_sponsor_class,collaborators,has_results
0,NCT00193635,CEL214,Pilot Study of Mycophenolate Mofetil in Congen...,Pilot Study of Mycophenolate Mofetil in Congen...,Northwell Health,OTHER,COMPLETED,2007-10,2002-03,2007-08,2005-09-09,2005-09-11,2005-09-19,2007-10-17,2007-10-19,False,Northwell Health,OTHER,Hoffmann-La Roche|University of Medicine and D...,False
1,NCT05348135,No:P.T.REC/012/003398,Combined Cognitive and Functional Strength Tra...,Synergistic Effect of Functional Strength Trai...,Cairo University,OTHER,COMPLETED,2022-04,2020-10-01,2022-03-11,2022-03-12,2022-04-20,2022-04-27,2022-04-20,2022-04-27,False,Cairo University,OTHER,NaN,False
2,NCT02576535,8844,Protocol for Staged Stereotactic Radiosurgery ...,Protocol for Staged Stereotactic Radiosurgery ...,NYU Langone Health,OTHER,COMPLETED,2023-11,2000-02,NaN,2015-10-12,2015-10-14,2015-10-15,2023-11-02,2023-11-07,False,NYU Langone Health,OTHER,NaN,True
3,NCT02084043,Synchro-Neb,In Vitro Assessment of a Breath-synchronized V...,In Vitro Comparison of Continuous and Breath-s...,"University Hospital St Luc, Brussels",OTHER,COMPLETED,2015-06,2014-03,2015-06,2014-03-08,2014-03-08,2014-03-11,2015-06-05,2015-06-08,False,"University Hospital St Luc, Brussels",OTHER,Université Catholique de Louvain|University of...,False
4,NCT05893043,GSBR-1290-03,A Study of Multiple Ascending Doses of GSBR-12...,"A Phase 1, Randomized, Double-blind, Placebo-c...","Gasherbrum Bio, Inc., a wholly owned subsidiar...",INDUSTRY,COMPLETED,2023-07,2023-04-24,2023-06-28,2023-05-30,2023-05-30,2023-06-07,2023-07-13,2023-07-14,False,"Gasherbrum Bio, Inc., a wholly owned subsidiar...",INDUSTRY,NaN,False


In [18]:
df.shape

(208000, 20)

In [ ]:
df2 = pd.read_excel("clinical_trials_separated_columns.xlsx")  

In [23]:
df2.shape

(354709, 20)